[![Open in SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/DEX-1101/sd-webui-sagemaker/blob/main/sd-webui-sagemaker.ipynb)
![Visitors](https://api.visitorbadge.io/api/visitors?path=https%3A%2F%2Fgithub.com%2FDEX-1101%2Fsd-webui-sagemaker%2Fblob%2Fmain%2Fsd-webui-sagemaker.ipynb&countColor=%23263759)

In [ ]:
# Run this cell ONLY ONCE.

%env TF_CPP_MIN_LOG_LEVEL=1

webui_path = "/home/studio-lab-user/content/x1101"
root_path = "/home/studio-lab-user/content"

import time
try:
    start_colab
except:
    start_colab = int(time.time())-5
    
!pip install -q git+https://github.com/DEX-1101/colablib
!pip install -q pytz
import os
import time
from colablib.colored_print import cprint, print_line
from colablib.utils import py_utils
start_time    = time.time()

print_line(0, color="green")
cprint("[+] Preparing Environment...", color="flat_cyan")
!wget -q -c https://raw.githubusercontent.com/DEX-1101/SecretNAI/main/template.txt -O {root_path}/download_list.txt
%conda install -q -y aria2
#%conda install -q -y libglib2.0-0
%conda install -q -y glib
%pip install -q opencv-python-headless huggingface-hub
!pip install -q torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1+cu118 torchtext==0.15.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118 -U
#!pip install -q xformers==0.0.20 triton==2.0.0 -U
!pip install -q xformers==0.0.18 triton==2.0.0 -U

print_line(0, color="green")
cprint("[+] Downloading Repository...", color="flat_cyan")
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/x1101/fix/resolve/main/zip/fastrepo_ext.zip -o fastrepo_ext.zip
cprint("Unpacking...", color="yellow")
!unzip -q -o fastrepo_ext.zip 
!rm fastrepo_ext.zip

print_line(0, color="green")
cprint("[+] Updating Repository...", color="flat_cyan")
%cd {webui_path}
!git pull &> /dev/null
!git reset --hard

print_line(0, color="green")
cprint("[+] Updating Extensions...", color="flat_cyan")
%cd {webui_path}/extensions
!find . -mindepth 1 -maxdepth 1 -type d -print -exec git -C {} pull \; &> /dev/null
cprint("All Extensions Updated !", color="flat_yellow")
!find . -mindepth 1 -maxdepth 1 -type d -print -exec git -C {} reset --hard \;

elapsed_time  = py_utils.calculate_elapsed_time(start_time)
print_line(0, color="green")
cprint(f"[+] All completed within {elapsed_time}.", color="flat_cyan")

In [ ]:
# Run Webu UI
# Get your ngrok_oken here : https://dashboard.ngrok.com/get-started/your-authtoken
ngrok_token = ""

#=================================================

!echo -n {start_colab} > {webui_path}/static/colabTimer.txt
%cd {webui_path}
!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' {webui_path}/modules/shared_options.py
!python launch.py --skip-torch-cuda-test --listen --xformers --enable-insecure-extension-access --ngrok {ngrok_token}

# use cpu
# !python launch.py --skip-torch-cuda-test --precision full --ngrok {ngrok_token} --no-half --use-cpu SD GFPGAN BSRGAN ESRGAN SCUNet CodeFormer --all

In [ ]:
# Download Model, LoRA, VAE, Embeddings, or Extension from Pastebin URL. Example usage : https://pastebin.com/ZPfcNx3u
# Or Open the download_list.txt and input link there on the corresponding #hastag.
# Put Hugging face token "hf_token" to download from private repository. Leave empty if NOT.
pastebin_url = ""
hf_token  = ""

#=================================================

import os
import time
from colablib.utils import py_utils
from pydantic import BaseModel
from colablib.utils.py_utils import get_filename
from colablib.sd_models.downloader import aria2_download, download
from colablib.colored_print import cprint, print_line
from colablib.utils.config_utils import read_config
from colablib.utils.git_utils import clone_repo

%store -r

root_dir            = "/home/studio-lab-user/content"
repo_dir            = os.path.join(root_dir, "x1101")
custom_model_url        = ""
custom_vae_url          = ""
custom_embedding_url    = ""
custom_LoRA_url         = ""
custom_extensions_url   = ""


models_dir          = os.path.join(repo_dir, "models", "Stable-diffusion")
vaes_dir            = os.path.join(repo_dir, "models", "VAE")
lora_dir            = os.path.join(repo_dir, "models", "Lora")
embeddings_dir      = os.path.join(repo_dir, "embeddings")
extensions_dir      = os.path.join(repo_dir, "extensions")
download_list       = os.path.join(root_dir, "download_list.txt")


class CustomDirs(BaseModel):
    url: str
    dst: str


user_header = f"Authorization: Bearer {hf_token}"

custom_dirs = {
    "model"       : CustomDirs(url=custom_model_url, dst=models_dir),
    "vae"         : CustomDirs(url=custom_vae_url, dst=vaes_dir),
    "embedding"   : CustomDirs(url=custom_embedding_url, dst=embeddings_dir),
    "lora"        : CustomDirs(url=custom_LoRA_url, dst=lora_dir),
    "extensions"  : CustomDirs(url=custom_extensions_url, dst=extensions_dir),

}

def parse_urls(filename):
    content = read_config(filename)
    lines   = content.strip().split('\n')
    result  = {}
    key     = ''
    for line in lines:
        if not line.strip():
            continue
        if line.startswith('//'):
            continue
        if line.startswith('#'):
            key = line[1:].lower()
            result[key] = []
        else:
            urls = [url.strip() for url in line.split(',') if url.strip() != '']
            result[key].extend(urls)
    return result


def get_filename(url, token=None):
    # Create headers dictionary with the token if provided
    headers = {}
    if token:
        headers['Authorization'] = f'Bearer {hf_token}'  # Assuming it's a Bearer token


def custom_download(custom_dirs):
    for key, value in custom_dirs.items():
        urls     = value.url.split(",")  # Split the comma-separated URLs
        dst      = value.dst

        if value.url:
            print_line(0, color="green")
            cprint(f" [+] Downloading {key}.", color="flat_yellow")

        for url in urls:
            url = url.strip()  # Remove leading/trailing whitespaces from each URL
            if url != "":
                print_line(0, color="green")
                if "|" in url:
                    url, filename = map(str.strip, url.split("|"))
                    if not filename.endswith((".safetensors", ".ckpt", ".pt", "pth")):
                        filename = filename + os.path.splitext(get_filename(url))[1]
                else:
                    if not url.startswith("fuse:"):
                        filename = get_filename(url)

                if url.startswith("fuse:"):
                    fuse(url, key, dst)
                elif key == "extensions":
                    clone_repo(url, cwd=dst)
                else:
                   download(url=url, filename=filename, user_header=user_header, dst=dst, quiet=False)



def download_from_textfile(filename):
    for key, urls in parse_urls(filename).items():
        key_lower = key.lower()
        if key_lower in custom_dirs:
            if custom_dirs[key_lower].url:
                custom_dirs[key_lower].url += ',' + ','.join(urls)
            else:
                custom_dirs[key_lower].url = ','.join(urls)
        else:
            cprint(f"Warning: Category '{key}' from the file is not found in custom_dirs.", color="flat_yellow")

def custom_download_list(url):
    filename = "custom_download_list.txt"
    filepath = os.path.join(root_dir, filename)
    if os.path.exists(filepath):
        os.remove(filepath)
    if 'pastebin.com' in url:
        if 'raw' not in url:
            url = url.replace('pastebin.com', 'pastebin.com/raw')
    download(url=url, filename=filename, user_header=user_header, dst=root_dir, quiet=True)
    return filepath

def main():
    start_time    = time.time()
    textfile_path = download_list
    if pastebin_url:
        textfile_path = custom_download_list(pastebin_url)
    download_from_textfile(textfile_path)
    custom_download(custom_dirs)

    elapsed_time  = py_utils.calculate_elapsed_time(start_time)
    print_line(0, color="green")
    cprint(f"[+] Download completed within {elapsed_time}.", color="flat_yellow")


main()

In [ ]:
# STORAGE INFO
!df -h

In [ ]:
# CLEAN STORAGAE if you need more space, will not delete your WEBUI Folder.

!pip cache purge
!conda clean --all -y
!rm -rf ~/.cache

%conda clean --all -y

%cd ~/
!pip freeze > reqs.txt
!pip uninstall -y -r reqs.txt

In [ ]:
# IF SOMETHING FEELS WRONG AND YOU CAN'T FIX IT, RUN THIS CELL, IT WILL DELETE ENTIRE NOTEBOOK,
# AND GUESS WHAT ! START FROM BEGINING AGAIN.
!rm -rf /home/studio-lab-user/